In [ ]:
import pathlib
import json
import helpers
from helpers import ai, apple
from bs4 import BeautifulSoup

import requests
import shutil

from urllib.parse import unquote

In [ ]:
NBS_DIR = pathlib.Path().resolve().parent
BASE_DIR = NBS_DIR
DATASET_DIR = BASE_DIR / "dataset"

In [ ]:
def clean_filename(filename):
    """
    Remove any query parameters 
    that may be included in the file name
    """
    if not isinstance(filename, str):
        return None
    if '?' in filename:
        cleaned_filename = filename.split('?', 1)[0]
    else:
        cleaned_filename = filename
    return cleaned_filename

def download_file(url, destination_path="."):
    fname = pathlib.Path(url).name 
    fname = clean_filename(fname)
    suffix = pathlib.Path(fname).suffix
    if suffix == ".mp3":
        fname = "audio.mp3"
    elif suffix == ".mp4":
        fname = "video.mp4"
    else:
        return 
    outpath = pathlib.Path(destination_path).resolve() / fname
    with requests.get(url,  stream=True) as r:
        with open(str(outpath), 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    return fname

In [ ]:
def convert_encoded_url(encoded_url):
    """
    Decode any encoded URL strings for
    episode downloading
    """
    decoded_url = unquote(encoded_url)
    return decoded_url

In [ ]:
for path in list(DATASET_DIR.glob("**/**/*.mp3"))[:5]:
    path.unlink()

In [ ]:
for path in list(DATASET_DIR.glob("**/**/*.mp4"))[:5]:
    path.unlink()

In [ ]:
for path in list(DATASET_DIR.glob("**/**/podcast.json"))[:5]:
    podcast_detail_dir = path.parent
    podcasts_dir = podcast_detail_dir.parent
    post_dir = podcasts_dir.parent
    post_id = post_dir.name
    podcast_data = json.loads(path.read_text())
    podcast_id = podcast_data.get('trackId')
    episode_url = podcast_data.get('episodeUrl')
    # print(episode_url, podcast_detail_dir)
    episode_url = convert_encoded_url(episode_url)
    print(download_file(episode_url, destination_path=podcast_detail_dir))
    # print(podcast_detail_dir)
    #